다음 노트북을 실행하려면, 아직 하지 않았다면 .env 파일에 `OPENAI_API_KEY`로 openai 키를 설정해야 합니다.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

다음으로, 임베딩 인덱스를 판다스 데이터프레임에 불러올 것입니다. 임베딩 인덱스는 `embedding_index_3m.json`이라는 JSON 파일에 저장되어 있습니다. 임베딩 인덱스에는 2023년 10월 말까지의 각 YouTube 자막에 대한 임베딩이 포함되어 있습니다.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

다음으로, `get_videos`라는 함수를 만들어서 쿼리에 대해 임베딩 인덱스를 검색할 것입니다. 이 함수는 쿼리와 가장 유사한 상위 5개의 비디오를 반환합니다. 함수의 동작 방식은 다음과 같습니다.

1. 먼저, 임베딩 인덱스의 복사본을 만듭니다.
2. 다음으로, OpenAI Embedding API를 사용해 쿼리의 임베딩을 계산합니다.
3. 그런 다음, 임베딩 인덱스에 `similarity`라는 새 컬럼을 만듭니다. `similarity` 컬럼에는 쿼리 임베딩과 각 비디오 세그먼트의 임베딩 간의 코사인 유사도가 들어갑니다.
4. 다음으로, 임베딩 인덱스를 `similarity` 컬럼을 기준으로 필터링합니다. 코사인 유사도가 0.75 이상인 비디오만 포함하도록 임베딩 인덱스를 필터링합니다.
5. 마지막으로, 임베딩 인덱스를 `similarity` 컬럼을 기준으로 정렬하고 상위 5개의 비디오를 반환합니다.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

이 함수는 매우 간단하며, 검색 쿼리의 결과를 출력하기만 합니다.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. 먼저, 임베딩 인덱스를 Pandas 데이터프레임에 불러옵니다.
2. 다음으로, 사용자에게 쿼리를 입력하라는 메시지가 표시됩니다.
3. 그런 다음 `get_videos` 함수를 호출하여 임베딩 인덱스에서 쿼리를 검색합니다.
4. 마지막으로, `display_results` 함수를 호출하여 결과를 사용자에게 보여줍니다.
5. 이후 사용자는 또 다른 쿼리를 입력하라는 메시지를 받게 됩니다. 사용자가 `exit`을 입력할 때까지 이 과정이 반복됩니다.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ko.png)

쿼리를 입력하라는 메시지가 표시됩니다. 쿼리를 입력하고 엔터를 누르세요. 애플리케이션은 쿼리와 관련된 비디오 목록을 반환합니다. 또한, 질문에 대한 답변이 위치한 비디오의 링크도 함께 제공합니다.

다음은 시도해 볼 수 있는 쿼리 예시입니다:

- Azure Machine Learning이란 무엇인가요?
- 합성곱 신경망은 어떻게 동작하나요?
- 신경망이란 무엇인가요?
- Azure Machine Learning에서 Jupyter 노트북을 사용할 수 있나요?
- ONNX란 무엇인가요?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서(원어)가 공식적인 기준임을 유의해 주시기 바랍니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 본 번역 사용으로 인해 발생하는 오해나 오역에 대해 당사는 책임을 지지 않습니다.
